# Analysis of new control run

`/local/projects/so_predict_additional/esm2m_froelicher/pi_control2` --> same variables as original control run

`/local/projects/so_predict_additional/esm2m_froelicher/pi_control` --> new limitation variables

[TOPAZ reference](https://data1.gfdl.noaa.gov/~arl/pubrel/r/mom4p1/src/ocean_shared/generic_tracers/generic_TOPAZ.F90)


In [5]:
import os
import warnings
warnings.filterwarnings("ignore", message="invalid value encountered in true_divide")
warnings.filterwarnings("ignore",message="Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range")
warnings.filterwarnings("ignore", message="invalid value encountered in reduce")

import xarray as xr
import numpy as np
import pickle as pkl

import matplotlib.pyplot as plt
import cmocean.cm as cm

%load_ext autoreload
%autoreload 2
import hmei.processing as pcs

In [8]:
rootdir = '/home/bbuchovecky/storage/so_predict_additional/esm2m_froelicher/pi_control2/'
writedir = '/home/bbuchovecky/storage/so_predict_additional_derived/CTRL/'

In [52]:
def comp_monthly_mean(
    ds, var, writedir, save=False):
    
    reg_masks = xr.open_dataset('/home/bbuchovecky/storage/so_predict_derived/regional_global_masks.nc')
    
    ## get ocean model grid information
    ocean_grid = xr.open_dataset('/local/projects/so_predict/esm2m_froelicher/GRID/ocean.static.nc')
    
    ## get grid-cell area for the ocean only
    total_area = xr.where(np.isnan(ds[var][0]), np.nan, ocean_grid['area_t'])
        
    regions = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']
    reg_mean = []

    for r in regions:
        area = total_area.where(reg_masks[r] == 1)
        area_sum = area.sum(dim={'xt_ocean', 'yt_ocean'})
        reg_var = ds[var].where(reg_masks[r] == 1)

        if var == 'SIE_area' or var == 'SIV_area':
            monthly_mean = reg_var.sum(dim={'xt_ocean', 'yt_ocean'})
        else:
            monthly_mean = (reg_var * ocean_grid['area_t']).sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum

        monthly_mean.name = r
        monthly_mean.attrs = ds[var].attrs
        monthly_mean.attrs['description'] = 'monthly mean'

        reg_mean.append(monthly_mean.copy(deep=True))

    monthly_mean = xr.merge(reg_mean)

    filename = var.lower()+'_ts_so_monthly_mean.nc'
    monthly_mean.attrs = {}
    monthly_mean.attrs['name'] = filename
        
    if save:
        monthly_mean.to_netcdf(writedir+'CTRL/'+var.upper()+'/'+filename)
        print(writedir+'CTRL/'+var.upper()+'/'+filename)
    if not save:
        print(writedir+'CTRL/'+var.upper()+'/'+filename)
        return monthly_mean

In [53]:
ocean_grid = xr.open_dataset('/local/projects/so_predict/esm2m_froelicher/GRID/ocean.static.nc')
# mld = xr.open_mfdataset(rootdir+'mld_*.nc').assign_coords({'xt_ocean':ocean_grid.xt_ocean})
comp_monthly_mean(mld, 'mld', writedir, save=False)

/home/bbuchovecky/storage/so_predict_additional_derived/CTRL/CTRL/MLD/mld_ts_so_monthly_mean.nc


<xarray.Dataset>
Dimensions:        (time: 6000)
Coordinates:
  * time           (time) object 2361-01-16 12:00:00 ... 2860-12-16 12:00:00
Data variables:
    SouthernOcean  (time) float32 dask.array<chunksize=(12,), meta=np.ndarray>
    Weddell        (time) float32 dask.array<chunksize=(12,), meta=np.ndarray>
    Indian         (time) float32 dask.array<chunksize=(12,), meta=np.ndarray>
    WestPacific    (time) float32 dask.array<chunksize=(12,), meta=np.ndarray>
    Ross           (time) float32 dask.array<chunksize=(12,), meta=np.ndarray>
    AmundBell      (time) float32 dask.array<chunksize=(12,), meta=np.ndarray>
Attributes:
    name:     mld_ts_so_monthly_mean.nc